In [ ]:
import torch
from torch import nn
import pandas as pd

In [ ]:
# Sequence length of input
WINDOW_SIZE = 14
# Number of hidden layer neurons in LSTM
HIDDEN_SIZE = 128
# Number of LSTMs
NUM_LSTMS = 4

In [ ]:
# Get your API key from https://www.alphavantage.co/support/#api-key
api_key = ''
symbol = 'IBM'
df = pd.read_csv(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&datatype=csv&apikey={api_key}')

In [ ]:
prices = df['close'].to_list()[:WINDOW_SIZE]
prices = prices[::-1]

In [ ]:
prices = torch.tensor(prices)
prices = prices.view((1, WINDOW_SIZE, 1))

In [ ]:
class LSTM_Model(nn.Module):
  def __init__(self, hidden_size, num_layers):
    super(LSTM_Model, self).__init__()

    self.relu = nn.ReLU()

    self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_size,
                        num_layers=num_layers, batch_first=True)
    
    self.fc1 = nn.Linear(hidden_size, 128)
    self.fc2 = nn.Linear(128, 1)


  def forward(self, x):
    x, _ = self.lstm(x)
    x = x[:, -1]
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [ ]:
model = LSTM_Model(HIDDEN_SIZE, NUM_LSTMS)
model.load_state_dict(torch.load("stock_model.pt", map_location=torch.device('cpu')))
model.eval()

LSTM_Model(
  (relu): ReLU()
  (lstm): LSTM(1, 128, num_layers=4, batch_first=True)
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

In [ ]:
with torch.no_grad():
  prediction = model(prices).item()

prediction

128.60043334960938